In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime

from funciones_ML import *
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay,classification_report,r2_score,RocCurveDisplay,mean_absolute_error,mean_squared_error,silhouette_score
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor, SGDClassifier, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.svm import SVC, SVR
import sklearn.cluster as cluster
from catboost import CatBoostClassifier
from catboost import Pool
import xgboost
from lightgbm import LGBMClassifier


# Configurar pandas para mostrar todas las filas y columnas
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


# Código para los tamaños de las fuentes y los "ticks" de los ejes:
plt.rc('font', size=6)
plt.rc('axes', labelsize=10, titlesize=10)
plt.rc('legend', fontsize=10)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [6]:
df_malware_total=pd.read_csv(r"D:\Cursos\REPOSITORIOS\DATASET\malware_total.csv")

C:\Users\victo\AppData\Local\Temp\ipykernel_19444\2968648179.py:1: DtypeWarning: Columns (7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_malware_total=pd.read_csv(r"D:\Cursos\REPOSITORIOS\DATASET\malware_total.csv")


In [3]:
df_malware_total.head()

,Id_session,Ip_host_org,Port_org,Ip_host_dest,Port_dest,Protocol_red,Service_protocol,Duration_conexion,Send_orig_bytes,receive_orig_bytes,...,Info_history_connection,Send_host_org_packs,Ip_orig_bytes,Resquest_receive_packs,Ip_dest_bytes,Route_tunnels_used,Malware_general_label,Malware_datail_label,Date,Time
0,CUmrqr4svHuSXJy5z7,192.168.100.103,51524.0,65.127.233.163,23.0,tcp,-,2.999051,0,0,...,S,3.0,180.0,0.0,0.0,-,Malicious,PartOfAHorizontalPortScan,2018-05-09,15:30:31.015810
1,CH98aB3s1kJeq6SFOc,192.168.100.103,56305.0,63.150.16.171,23.0,tcp,-,-,-,-,...,S,1.0,60.0,0.0,0.0,-,Malicious,PartOfAHorizontalPortScan,2018-05-09,15:30:31.025054
2,C3GBTkINvXNjVGtN5,192.168.100.103,41101.0,111.40.23.49,23.0,tcp,-,-,-,-,...,S,1.0,60.0,0.0,0.0,-,Malicious,PartOfAHorizontalPortScan,2018-05-09,15:30:31.045044
3,CDe43c1PtgynajGI6,192.168.100.103,60905.0,131.174.215.147,23.0,tcp,-,2.998796,0,0,...,S,3.0,180.0,0.0,0.0,-,Malicious,PartOfAHorizontalPortScan,2018-05-09,15:30:32.016239
4,CJaDcG3MZzvf1YVYI4,192.168.100.103,44301.0,91.42.47.63,23.0,tcp,-,-,-,-,...,S,1.0,60.0,0.0,0.0,-,Malicious,PartOfAHorizontalPortScan,2018-05-09,15:30:32.024985


In [9]:
#CREAR UNA COLUMNA CON INTERVARIO HORARIO, DIA DE LA SEMANA Y  Y PASAR LA HORA A EN PUNTO, PARA FACILITAR EL PROCESO

# Convertir la columna 'Time' a formato datetime
df_malware_total['Time'] = pd.to_datetime(df_malware_total['Time'])

# Crear la columna 'Time_clock'
df_malware_total['Time_clock'] = df_malware_total['Time'].dt.hour

# Crear la columna 'intervalo'
bins = [0, 3, 6, 9, 12, 15, 18, 21, 24]
labels = ['0-3', '3-6', '6-9', '9-12', '12-15', '15-18', '18-21', '21-0']

df_malware_total['Interval_time'] = pd.cut(df_malware_total['Time_clock'], bins=bins, labels=labels, include_lowest=True, right=False)



C:\Users\victo\AppData\Local\Temp\ipykernel_19444\114976531.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_malware_total['Time'] = pd.to_datetime(df_malware_total['Time'])


In [10]:

# Convierte la columna "Date" a tipo Date
df_malware_total["Date"] = pd.to_datetime(df_malware_total["Date"])

df_malware_total["Anyo"] = df_malware_total["Date"].dt.year

# Extrae el día de la semana usando la función dt.dayofweek
df_malware_total["Month"] = df_malware_total["Date"].dt.month

# Crear la columna 'Day_week'
df_malware_total['Day_week'] = df_malware_total['Date'].dt.dayofweek

# Mostrar el DataFrame resultante
df_malware_total.head()


,Id_session,Ip_host_org,Port_org,Ip_host_dest,Port_dest,Protocol_red,Service_protocol,Duration_conexion,Send_orig_bytes,receive_orig_bytes,...,Route_tunnels_used,Malware_general_label,Malware_datail_label,Date,Time,Time_clock,Interval_time,Anyo,Month,Day_week
0,CUmrqr4svHuSXJy5z7,192.168.100.103,51524.0,65.127.233.163,23.0,tcp,-,2.999051,0,0,...,-,Malicious,PartOfAHorizontalPortScan,2018-05-09,2024-02-03 15:30:31.015810,15,15-18,2018,5,2
1,CH98aB3s1kJeq6SFOc,192.168.100.103,56305.0,63.150.16.171,23.0,tcp,-,-,-,-,...,-,Malicious,PartOfAHorizontalPortScan,2018-05-09,2024-02-03 15:30:31.025054,15,15-18,2018,5,2
2,C3GBTkINvXNjVGtN5,192.168.100.103,41101.0,111.40.23.49,23.0,tcp,-,-,-,-,...,-,Malicious,PartOfAHorizontalPortScan,2018-05-09,2024-02-03 15:30:31.045044,15,15-18,2018,5,2
3,CDe43c1PtgynajGI6,192.168.100.103,60905.0,131.174.215.147,23.0,tcp,-,2.998796,0,0,...,-,Malicious,PartOfAHorizontalPortScan,2018-05-09,2024-02-03 15:30:32.016239,15,15-18,2018,5,2
4,CJaDcG3MZzvf1YVYI4,192.168.100.103,44301.0,91.42.47.63,23.0,tcp,-,-,-,-,...,-,Malicious,PartOfAHorizontalPortScan,2018-05-09,2024-02-03 15:30:32.024985,15,15-18,2018,5,2


In [12]:
df_malware_total.Day_week.unique()

array([2, 3, 4, 5, 6, 0, 1])

In [13]:
df_malware_total.to_csv(r"D:\Cursos\REPOSITORIOS\DATASET\malware_total.csv")

In [14]:
df_malware_total.head()

,Id_session,Ip_host_org,Port_org,Ip_host_dest,Port_dest,Protocol_red,Service_protocol,Duration_conexion,Send_orig_bytes,receive_orig_bytes,...,Route_tunnels_used,Malware_general_label,Malware_datail_label,Date,Time,Time_clock,Interval_time,Anyo,Month,Day_week
0,CUmrqr4svHuSXJy5z7,192.168.100.103,51524.0,65.127.233.163,23.0,tcp,-,2.999051,0,0,...,-,Malicious,PartOfAHorizontalPortScan,2018-05-09,2024-02-03 15:30:31.015810,15,15-18,2018,5,2
1,CH98aB3s1kJeq6SFOc,192.168.100.103,56305.0,63.150.16.171,23.0,tcp,-,-,-,-,...,-,Malicious,PartOfAHorizontalPortScan,2018-05-09,2024-02-03 15:30:31.025054,15,15-18,2018,5,2
2,C3GBTkINvXNjVGtN5,192.168.100.103,41101.0,111.40.23.49,23.0,tcp,-,-,-,-,...,-,Malicious,PartOfAHorizontalPortScan,2018-05-09,2024-02-03 15:30:31.045044,15,15-18,2018,5,2
3,CDe43c1PtgynajGI6,192.168.100.103,60905.0,131.174.215.147,23.0,tcp,-,2.998796,0,0,...,-,Malicious,PartOfAHorizontalPortScan,2018-05-09,2024-02-03 15:30:32.016239,15,15-18,2018,5,2
4,CJaDcG3MZzvf1YVYI4,192.168.100.103,44301.0,91.42.47.63,23.0,tcp,-,-,-,-,...,-,Malicious,PartOfAHorizontalPortScan,2018-05-09,2024-02-03 15:30:32.024985,15,15-18,2018,5,2


In [17]:
len(df_malware_total.columns)

29

In [15]:
#CREAMOS NUEVO DATAFRAME ORDENADO

columnas_elegidas = ['Anyo', 'Month', 'Day_week','Time_clock', 'Interval_time', 'Port_org','Port_dest','missed_bytes',
       'Send_host_org_packs', 'Ip_orig_bytes','Resquest_receive_packs', 'Ip_dest_bytes', 
        'Id_session', 'Ip_host_org','Ip_host_dest','Protocol_red', 'Service_protocol', 'Duration_conexion',
       'Send_orig_bytes', 'receive_orig_bytes', 'Status_connect','Info_history_connection',"Malware_general_label",'Malware_datail_label',
       'Lost_orig_bytes', 'Lost_dest_bytes','Route_tunnels_used']

# Crear un nuevo DataFrame con las columnas seleccionadas y en el orden deseado
df_malware_total_orden= df_malware_total[columnas_elegidas].copy()

In [22]:
df_malware_total_orden

,Anyo,Month,Day_week,Time_clock,Interval_time,Port_org,Port_dest,missed_bytes,Send_host_org_packs,Ip_orig_bytes,Resquest_receive_packs,Ip_dest_bytes,Id_session,Ip_host_org,Ip_host_dest,Protocol_red,Service_protocol,Duration_conexion,Send_orig_bytes,receive_orig_bytes,Status_connect,Info_history_connection,Malware_general_label,Malware_datail_label,Lost_orig_bytes,Lost_dest_bytes,Route_tunnels_used
0,2018,5,2,15,15-18,51524.0,23.0,0.0,3.0,180.0,0.0,0.0,CUmrqr4svHuSXJy5z7,192.168.100.103,65.127.233.163,tcp,-,2.999051,0,0,S0,S,Malicious,PartOfAHorizontalPortScan,-,-,-
1,2018,5,2,15,15-18,56305.0,23.0,0.0,1.0,60.0,0.0,0.0,CH98aB3s1kJeq6SFOc,192.168.100.103,63.150.16.171,tcp,-,-,-,-,S0,S,Malicious,PartOfAHorizontalPortScan,-,-,-
2,2018,5,2,15,15-18,41101.0,23.0,0.0,1.0,60.0,0.0,0.0,C3GBTkINvXNjVGtN5,192.168.100.103,111.40.23.49,tcp,-,-,-,-,S0,S,Malicious,PartOfAHorizontalPortScan,-,-,-
3,2018,5,2,15,15-18,60905.0,23.0,0.0,3.0,180.0,0.0,0.0,CDe43c1PtgynajGI6,192.168.100.103,131.174.215.147,tcp,-,2.998796,0,0,S0,S,Malicious,PartOfAHorizontalPortScan,-,-,-
4,2018,5,2,15,15-18,44301.0,23.0,0.0,1.0,60.0,0.0,0.0,CJaDcG3MZzvf1YVYI4,192.168.100.103,91.42.47.63,tcp,-,-,-,-,S0,S,Malicious,PartOfAHorizontalPortScan,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25010998,2018,7,3,8,6-9,28057.0,23.0,0.0,1.0,40.0,0.0,0.0,CIHonx2ymqlX5DqnU2,192.168.100.111,173.94.58.125,tcp,-,-,-,-,S0,S,Malicious,PartOfAHorizontalPortScan,-,-,-
25010999,2018,7,3,8,6-9,52876.0,23.0,0.0,8.0,360.0,1.0,44.0,CwhtUD3BIdOP5L6Y8g,192.168.100.111,69.196.96.231,tcp,-,209.874367,0,0,S1,ShA,Benign,-,-,-,-
25011000,2018,7,3,7,6-9,39234.0,23.0,0.0,5.0,220.0,1.0,44.0,CxHWyM2vp5x1mCZmp3,192.168.100.111,192.121.45.63,tcp,-,165.760290,0,0,S1,ShA,Benign,-,-,-,-
25011001,2018,7,3,8,6-9,11.0,0.0,0.0,1.0,68.0,0.0,0.0,CtmaRjXW7FOus0sPa,168.102.14.4,192.168.100.111,icmp,-,-,-,-,OTH,-,Benign,-,-,-,-


In [18]:
len(df_malware_total_orden.columns)

27

In [19]:
df_malware_total_orden.to_csv(r"D:\Cursos\REPOSITORIOS\DATASET\malware_total_orden.csv", index=False)

In [23]:
df_malware_total_orden_1 = df_malware_total_orden.copy()
# Características (X)
# X consiste en una lista de cadenas de texto
X = df_malware_total_orden_1['Id_session']
kmeans = cluster.KMeans(n_clusters=3)

# Vectorizamos el texto usando TfidfVectorizer y  con vectorizer transformamos el conjunto de datos  a una matriz de valores que representan la frecuencia de cada palabra en cada documento(X_Tf)
vectorizer = TfidfVectorizer()
X_Tf = vectorizer.fit_transform(X)
# Determinamos el número óptimo de clústeres usando K-means, la cual analiza la homogeneidad de los datos , cuanto mas homogeneos menor es el rangpo a establecer. 
posibles_valores_clusters = range(2, 11)  # el rango se amplia o se reduce dependiendo de la posble homegenidad de los datos
for k in posibles_valores_clusters:
    #iteramos sobre el rango de valores de los posibles clústeres. 
    #Para cada valor, el algoritmo K-Means se ejecuta y se calcula el índice de silueta.
    # EL  índice de silueta, es una medida de la compactibilidad de un clúster y su separación con respecto a otros clústeres, y   mayor indice se considera el número de clústeres óptimo.
    lista_clusters = []
    agrupamiento = kmeans(n_clusters=k, random_state=42)
    agrupamiento.fit(X_Tf)
    labels = agrupamiento.labels_
    lista_clusters.append(silhouette_score(X_Tf, labels))

# Verificar la longitud del DataFrame df_malware_total_orden_1
n_filas = len(df_malware_total_orden_1)

# Crear un bucle para asignar la etiqueta clúster a cada elemento del DataFrame df_malware_total_orden_1
for i in range(min(n_filas, len(lista_clusters))):
  # Asignar la etiqueta de clúster correspondiente al elemento i del DataFrame df_malware_total_orden_1
  df_malware_total_orden_1.loc[i, 'Cluster'] = lista_clusters[i]
   
# Elegir el número óptimo de clústeres (aquí, el número con el mayor índice de silueta)
num_clusters = posibles_valores_clusters[lista_clusters.index(max(lista_clusters))]

# Entrenar K-means con el número óptimo de clústeres
agrupamiento = kmeans(n_clusters=num_clusters, random_state=42)
agrupamiento.fit(X_Tf)

# Etiquetar las muestras como 'normal' o 'anómala' según la distancia al centroide
df_malware_total_orden_1['Etiqueta'] = agrupamiento.predict(X_Tf)
df_malware_total_orden_1['Etiqueta'] = df_malware_total_orden_1['Etiqueta'].apply(lambda x: 'anormal' if x == 1 else 'normal')# funcion anonima que dice que si x=1 es anomala y sino es normal

#se crean 2 columnas  nuevas cluster y Etiaueta
df_malware_total_orden_1.head(1)

TypeError: 'KMeans' object is not callable

In [ ]:

#funcion para reducir la dimensionalidad de un vector de tamaño 40000 a una variable que espera un vector de tamaño 1.
def reducir_dimensionalidad(X, n_columnas=2):
    # Calculamos las medias de las columnas de los datos en el eje de las filas
    medias = X.mean(axis=0)

    # Calculamos las desviaciones estándar de las columnas de los datos
    desviaciones_estandar = X.std(axis=0)

    # Normalizamos los datos, ya que al restar la media de cada columna, se centra la columna en 0, es decir la media de la columna ahora es 0 y
    # al dividir cada columna por la desviación estándar de esa columna, se obtiene que cada  columna seleccionada  tenga una varianza de 1.
    # normalizando con todo ello y todos los datos qyengan la misma importancia
    X_norm = (X - medias) / desviaciones_estandar

    # Usamos la funcion de numpy argsort para seleccionar los  columnas(n_columnas) con mayor varianza, 
    #ya que Las columnas con mayor varianza son las que tienen los datos más dispersos, por lo tanto mayor cantidad de información posible. la 
    columnas_seleccionadas = np.argsort(X_norm.var(axis=0))[-n_columnas:]# esto ultimo devuelve de la lista los valores de mayor varianza

    # Devolvemos las  columnas(n_columnas) seleccionadas
    return X_norm[:, columnas_seleccionadas]


# Seleccionar la columna de interés
X_norm = df_malware_total_orden_1['Id_session']

# Vectorizar el texto usando TfidfVectorizer
vectorizer = TfidfVectorizer()
X_tf = vectorizer.fit_transform(X)

# Aplicar k-means para agrupar los textos
n_clusters = 5  # Puedes ajustar este valor según tu necesidad
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df_malware_total_orden_1['Cluster'] = kmeans.fit_predict(X_tf)

#mapeo las etiquetas de clúster (c) para que scatter, asignadole lo qu espera no valores numericos sino  colores 

def colores_clusters(cluster):
    if cluster == 0:
        return "red"
    elif cluster == 1:
        return "green"
    elif cluster == 2:
        return "blue"
    else:
        return "yellow"

# convierto la matriz _matrix a matriz_array, por ser mas faciles de trabajar
X_tf_ndarray = X_tf.toarray()
#reduzco con la funcion numpy squeenze, la diemnsion primera de una matriz, en nuestro caso (40000, dimesion) eleimina dimension quedansoe con el numero de valores solo
x_reducido = X_tf_ndarray[:, 0].squeeze()
y_reducido = X_tf_ndarray[:, 1].squeeze()

#pintamos
plt.figure(figsize=(12,6))
plt.scatter(x_reducido, y_reducido, c=[colores_clusters(cluster) for cluster in df_malware_total_orden_1['Cluster']], cmap='magma', alpha=0.4, s=500, label='Anomalías')
# Cambiar el color del fondo
plt.style.use('seaborn-darkgrid')
#titulo y etiqyuetas
plt.title("Gráfico densidad 'Resultado_analisis' para conocer homegneidad de datos y detectar patrones anómalos")
plt.xlabel('Dimensión 1_ uso de tecnicas de modelado de anomalías o aprendizaje automático para entender mejor la anomalía y predecir futuras anomalías')
plt.ylabel('Dimensión 2')
plt.legend()
plt.show()

In [31]:
df_malware_total_orden.Malware_datail_label.unique()

array(['PartOfAHorizontalPortScan', '-', 'C&C', 'Torii', 'Attack', nan,
       'FileDownload', 'HeartBeat'], dtype=object)

In [27]:
df_malware_total_orden.Malware_general_label.unique()

array(['Malicious', 'Benign', 'Malicious   C&C',
       'Malicious   PartOfAHorizontalPortScan', 'Malicious   DDoS',
       'Malicious   Attack', 'Malicious   FileDownload'], dtype=object)

In [30]:
df_malware_total_orden.Malware_general_label.head(100)

0     Malicious
1     Malicious
2     Malicious
3     Malicious
4     Malicious
        ...    
95    Malicious
96       Benign
97    Malicious
98    Malicious
99       Benign
Name: Malware_general_label, Length: 100, dtype: object

In [35]:
obtener_estadisticas(df_malware_total_orden)

,porcentaje_cardinalidad,Tipo,media,moda,std,var,Q1,mediana,Q3,Categoria,porcentaje_NaN
Anyo,0.0,int32,2018.28,No,0.45,0.2,2018.0,2018.0,2019.0,numerica discreta,0.0
Month,0.0,int32,8.66,No,3.37,11.36,7.0,9.0,12.0,numerica discreta,0.0
Day_week,0.0,int32,3.81,No,1.08,1.17,3.0,4.0,5.0,numerica discreta,0.0
Time_clock,0.0,int32,12.96,No,7.85,61.6,5.0,14.0,21.0,numerica continua,0.0
Interval_time,0.0,category,No,21-0,No,No,No,No,No,categorica ordinal,0.0
Port_org,0.26,float64,39909.62,No,16091.97,258951549.59,32980.0,42993.0,52360.0,numerica continua,0.0
Port_dest,0.26,float64,9710.51,No,22174.48,491707739.75,23.0,23.0,81.0,numerica continua,0.0
missed_bytes,0.0,float64,76.33,No,381679.91,145679555881.049988,0.0,0.0,0.0,numerica continua,0.0
Send_host_org_packs,0.0,float64,13.5,No,17511.7,306659531.76,1.0,1.0,3.0,numerica continua,0.0
Ip_orig_bytes,0.01,float64,590.02,No,687810.32,473083036333.98999,40.0,60.0,180.0,numerica continua,0.0


# BOOSTING

In [33]:
df_malware_total_orden.columns

Index(['Anyo', 'Month', 'Day_week', 'Time_clock', 'Interval_time', 'Port_org',
       'Port_dest', 'missed_bytes', 'Send_host_org_packs', 'Ip_orig_bytes',
       'Resquest_receive_packs', 'Ip_dest_bytes', 'Id_session', 'Ip_host_org',
       'Ip_host_dest', 'Protocol_red', 'Service_protocol', 'Duration_conexion',
       'Send_orig_bytes', 'receive_orig_bytes', 'Status_connect',
       'Info_history_connection', 'Malware_general_label',
       'Malware_datail_label', 'Lost_orig_bytes', 'Lost_dest_bytes',
       'Route_tunnels_used'],
      dtype='object')

In [32]:
X=df_malware_total[['Anyo', 'Month', 'Day_week', 'Time_clock', 'Interval_time', 'Port_org',
       'Port_dest', 'missed_bytes', 'Send_host_org_packs', 'Ip_orig_bytes',
       'Resquest_receive_packs', 'Ip_dest_bytes', 'Id_session', 'Ip_host_org',
       'Ip_host_dest', 'Protocol_red', 'Service_protocol', 'Duration_conexion',
       'Send_orig_bytes', 'receive_orig_bytes', 'Status_connect',
       'Info_history_connection', 'Malware_general_label',
       'Malware_datail_label', 'Lost_orig_bytes', 'Lost_dest_bytes',
       'Route_tunnels_used']]
y= df_malware_total[ 'Malware_general_label']

VOY A USAR EL METODO LABERENCONDER PARA PASAR LA CATEGORIGAS ORDINALES A NUMERICAS Y EL ONE HOT ENCONDER PARA LA NOMINALES

In [36]:
columnas_cat_ordinales=['Interval_time','Id_session', 'Ip_host_org',
       'Ip_host_dest', 'Protocol_red', 'Service_protocol', 'Duration_conexion',
       'Send_orig_bytes', 'receive_orig_bytes', 'Status_connect',
       'Info_history_connection', 'Malware_general_label',
       'Malware_datail_label']
columnas_num_continuas=['Time_clock', 'Port_org','Port_dest', 'missed_bytes', 'Send_host_org_packs', 'Ip_orig_bytes',
       'Resquest_receive_packs', 'Ip_dest_bytes'
       'Resquest_receive_packs', 'Ip_dest_bytes']
xolumnas_num_discretas=['Anyo', 'Month', 'Day_week', ]
columnas_cat_nominales=['Lost_orig_bytes', 'Lost_dest_bytes',
       'Route_tunnels_used']

PROCESO DE TRANSFORMACION Y PREPARACION DE DATOS

In [40]:
# PARA LAS COLUMNAS CATEGORICAS ORDINALES
label_encoder = LabelEncoder()

# Convertir la columna completa a tipo de datos str
for columna in columnas_cat_ordinales:
    df_malware_total_orden_1[columna] = df_malware_total_orden_1[columna].astype(str)

for columna in columnas_cat_ordinales:
    df_malware_total_orden_1[columna] = label_encoder.fit_transform(df_malware_total_orden_1[columna])

In [42]:
df_malware_total_orden_1.head()

,Anyo,Month,Day_week,Time_clock,Interval_time,Port_org,Port_dest,missed_bytes,Send_host_org_packs,Ip_orig_bytes,Resquest_receive_packs,Ip_dest_bytes,Id_session,Ip_host_org,Ip_host_dest,Protocol_red,Service_protocol,Duration_conexion,Send_orig_bytes,receive_orig_bytes,Status_connect,Info_history_connection,Malware_general_label,Malware_datail_label,Lost_orig_bytes,Lost_dest_bytes,Route_tunnels_used
0,2018,5,2,15,2,51524.0,23.0,0.0,3.0,180.0,0.0,0.0,2686887,20264,11379722,1,0,454125,1,1,6,41,1,5,0,0,0
1,2018,5,2,15,2,56305.0,23.0,0.0,1.0,60.0,0.0,0.0,21586007,20264,11281919,1,0,0,0,0,6,41,1,5,0,0,0
2,2018,5,2,15,2,41101.0,23.0,0.0,1.0,60.0,0.0,0.0,3469427,20264,8681071,1,0,0,0,0,6,41,1,5,0,0,0
3,2018,5,2,15,2,60905.0,23.0,0.0,3.0,180.0,0.0,0.0,20015319,20264,2747822,1,0,453870,1,1,6,41,1,5,0,0,0
4,2018,5,2,15,2,44301.0,23.0,0.0,1.0,60.0,0.0,0.0,22677043,20264,1317443,1,0,0,0,0,6,41,1,5,0,0,0


In [45]:
# para las columnas categoricas nominales
df_malware_orden_total_1 = pd.get_dummies(df_malware_total_orden_1, columns=columnas_cat_nominales)

In [46]:
df_malware_total_orden_1

,Anyo,Month,Day_week,Time_clock,Interval_time,Port_org,Port_dest,missed_bytes,Send_host_org_packs,Ip_orig_bytes,Resquest_receive_packs,Ip_dest_bytes,Id_session,Ip_host_org,Ip_host_dest,Protocol_red,Service_protocol,Duration_conexion,Send_orig_bytes,receive_orig_bytes,Status_connect,Info_history_connection,Malware_general_label,Malware_datail_label,Lost_orig_bytes,Lost_dest_bytes,Route_tunnels_used
0,2018,5,2,15,2,51524.0,23.0,0.0,3.0,180.0,0.0,0.0,2686887,20264,11379722,1,0,454125,1,1,6,41,1,5,0,0,0
1,2018,5,2,15,2,56305.0,23.0,0.0,1.0,60.0,0.0,0.0,21586007,20264,11281919,1,0,0,0,0,6,41,1,5,0,0,0
2,2018,5,2,15,2,41101.0,23.0,0.0,1.0,60.0,0.0,0.0,3469427,20264,8681071,1,0,0,0,0,6,41,1,5,0,0,0
3,2018,5,2,15,2,60905.0,23.0,0.0,3.0,180.0,0.0,0.0,20015319,20264,2747822,1,0,453870,1,1,6,41,1,5,0,0,0
4,2018,5,2,15,2,44301.0,23.0,0.0,1.0,60.0,0.0,0.0,22677043,20264,1317443,1,0,0,0,0,6,41,1,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25010998,2018,7,3,8,6,28057.0,23.0,0.0,1.0,40.0,0.0,0.0,22096242,20266,6058221,1,0,0,0,0,6,41,1,5,0,0,0
25010999,2018,7,3,8,6,52876.0,23.0,0.0,8.0,360.0,1.0,44.0,15203388,20266,11611916,1,0,455232,1,1,7,50,0,0,0,0,0
25011000,2018,7,3,7,6,39234.0,23.0,0.0,5.0,220.0,1.0,44.0,15460843,20266,7353903,1,0,267903,1,1,7,50,0,0,0,0,0
25011001,2018,7,3,8,6,11.0,0.0,0.0,1.0,68.0,0.0,0.0,13893575,16471,7369596,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
df_malware_total_orden_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25011003 entries, 0 to 25011002
Data columns (total 27 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Anyo                     int32  
 1   Month                    int32  
 2   Day_week                 int32  
 3   Time_clock               int32  
 4   Interval_time            int32  
 5   Port_org                 float64
 6   Port_dest                float64
 7   missed_bytes             float64
 8   Send_host_org_packs      float64
 9   Ip_orig_bytes            float64
 10  Resquest_receive_packs   float64
 11  Ip_dest_bytes            float64
 12  Id_session               int32  
 13  Ip_host_org              int32  
 14  Ip_host_dest             int32  
 15  Protocol_red             int32  
 16  Service_protocol         int32  
 17  Duration_conexion        int32  
 18  Send_orig_bytes          int32  
 19  receive_orig_bytes       int32  
 20  Status_connect           int32  
 21  Info_h

In [48]:
funcion_categorias(df_malware_total_orden_1)

,Categoria
Anyo,numerica discreta
Month,numerica discreta
Day_week,numerica discreta
Time_clock,numerica continua
Interval_time,numerica discreta
Port_org,numerica continua
Port_dest,numerica continua
missed_bytes,numerica continua
Send_host_org_packs,numerica continua
Ip_orig_bytes,numerica continua


In [55]:
df_malware_total_orden_1.to_csv(r"D:\Cursos\REPOSITORIOS\DATASET\df_malware_total_enconding", index=False)

HAGO EL SPLIT

In [51]:
# monto el grupo de entranimento y de test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"el numero de elementos de elementos de X_train es: {len(X_train)}")
print(f"el numero de elementos de elementos de y_train es: {len(y_train)}")
print(f"el numero de elementos de elementos de X_test es: {len(X_test)}") 
print(f"el numero de elementos de elementos de y_test es: {len(y_test)}")

el numero de elementos de elementos de X_train es: 20008802
el numero de elementos de elementos de y_train es: 20008802
el numero de elementos de elementos de X_test es: 5002201
el numero de elementos de elementos de y_test es: 5002201


**XGBoost** (la target sera 'Malware_general_label', para que clasificque en bign o malicious)

In [52]:
xgb_clas = xgboost.XGBClassifier(random_state=42)

xgb_clas.fit(X_train, y_train)
y_pred = xgb_clas.predict(X_test)
print(classification_report(y_test, y_pred))

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5 6], got ['Benign' 'Malicious' 'Malicious   Attack' 'Malicious   C&C'
 'Malicious   DDoS' 'Malicious   FileDownload'
 'Malicious   PartOfAHorizontalPortScan']

**LGBMClassifier**

In [53]:

lgbm_clas = LGBMClassifier(random_state=42)

lgbm_clas.fit(X_train, y_train, categorical_feature= y)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

**CatboosT**

In [54]:


cb = CatBoostClassifier(iterations = 200,
                        learning_rate = 0.9,
                        verbose = False)
pool_train = Pool(X_train, y_train)
pool_test = Pool(X_test)
cb.fit(pool_train)

CatBoostError: features data: pandas.DataFrame column 'Interval_time' has dtype 'category' but is not in  cat_features list

In [ ]:
y_pred_cat = cb.predict(pool_test)
print(classification_report(y_test,y_pred_cat))